<a href="https://colab.research.google.com/github/Talha-Bicak/GarbageClassification/blob/main/GarbageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install imutils

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import urllib
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import random , os , glob
from imutils import paths
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from urllib.request import urlopen

#Warningleri kapatmak için kullanılmaktadır.

import warnings
warnings.filterwarnings('ignore')

#Model değerlendirme için kullanılacak olan kütüphaneler
from sklearn.metrics import confusion_matrix, classification_report

# Model için kullanılacak olan kütüphaneler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, MaxPool2D, Dense, Dropout, SpatialDropout2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
dir_path = '/content/drive/MyDrive/GarbageClassification/Garbage classification'

In [ ]:
# Target_Size ve Label Değerlerinin Belirlenmesi

target_size = (224, 224)

waste_labels = {"cardboard": 0,
                "glass": 1,
                "metal" : 2,
                "paper" : 3,
                "plastic" : 4,
                "trash" : 5}

In [ ]:
def load_datasets(path):
    """
Görsellerin bulunduğu dizindeki görüntüyü okuyup etiketlerini oluşturur.
Parametreler:
path: Görsellerin bulunduğu dizini ifade eder.
Return:
x: Görüntülere ait matris bilgilerini tutar.
labels: Görüntünün ait olduğu sınıf bilgisini tutan liste.
    """


    x = []
    labels = []

    # Gönderdiğimiz pathdeki görüntüleri listeleyip sıralamaktadır.
    image_paths = sorted(list(paths.list_images(path)))

    for image_path in image_paths:
        # Belirtilen pathdeki görüntüler openCV kütüphanesi ile okunmaktadır.
        img = cv2.imread(image_path)

        # Okunan görüntüler başlangıçta belirlenen target_size'a göre yeniden ölçeklendirilir.
        img = cv2.resize(img, target_size)

        # Ölçeklendirilen görüntüler x listesine eklenir.
        x.append(img)

        # Her bir path "/" ifadesi ile ayrıldığında dönen listenin sonran ikinci elamanı label'ı temsil etmektedir.
        label = image_path.split(os.path.sep)[-2]

        # Yakalanan labelların sayısal değer karşılıklarının olduğu waste_labels sözlüğü içerisinden gönderilen key
        # değerine karşılık value değeri alınarak label oluşturulur.
        labels.append(waste_labels[label])

    # Veri seti random bir şekilde karıştırılır.
    x, labels = shuffle(x, labels, random_state=42)

    # Boyut ve Sınıf bilgisi raporlanmaktadır.
    print(f"X boyutu: {np.array(x).shape}")
    print(f"Label sınıf sayısı: {len(np.unique(labels))} Gözlem sayısı: {len(labels)}")

    return x, labels

In [ ]:
x, labes = load_datasets(dir_path)

KeyboardInterrupt: ignored

In [ ]:
input_shape = (cv2.imread(dir_path)).shape

AttributeError: ignored

In [ ]:
def visualize_img(image_batch, label_batch):
"""

Veri seti içerisinden görüntü görselleştirir.

Parametreler:

image_batch: Görüntülere ait matris bilgilerini tutar.

label_batch: Görüntünün ait olduğu sınıf bilgisini tutan liste.
"""
  plt.figure(figsize=(10,10))
  for n in range(10):
    ax = plt.subplot(5,5,n+1)
    plt.imshow(image_batch[n])
    plt.title(np.array(list(waste_labels.keys()))[to_categorical(labels, num_classes=6)[n]==1][0].title())
    plt.axis('off')

In [ ]:
# Train veri seti için bir generator tanımlayalım.
train = ImageDataGenerator(horizontal_flip = True, #Görüntüleri rastgele yatay olarak çevirmektedir.
                           vertical_flip = True,   #Görüntüleri rastgele dikey olarak çevirmektedir.
                           validation_split = 0.1, #Test veri setinin oranı
                           rescale = 1./255,       #Görüntüleri yeniden ölçeklendirme işlemi yapar.Buradaki oran verileri 0-1 arasına dönüştürür.
                           shear_range = 0.1,      #Görüntülere eğim vererek saat yönünün tersine yeni veriler üretmemizi sağlar.
                           zoom_range = 0.1,
                           width_shift_range = 0.1,#Genişlik kaydırma için kullanılır.
                           height_shift_range = 0.1)#Uzunluk kaydırma işlemi için kullanılır.

# Test veri seti için bir generator tanımlayalım.
test = ImageDataGenerator(rescale= 1/255,
                          validation_split = 0.1)

In [ ]:
train_generator = train.flow_from_directory(directory=dir_path,         #Veri Setimizin bulunduğu dizin.
                                            target_size= (target_size), #Görüntülerin boyut bilgisini içerir.
                                            class_mode = "categorical", #
                                            subset = 'training')
test_genertor = test.flow_from_directory(directory= dir_path,
                                         target_size= (target_size),
                                         batch_size=251,               #Verilerin boyutunu ifade eder.
                                         class_mode = 'categorical',   #İkiden fazla kategoride veri bulunduğu zaman kullanılır.Eğerki iki farklı verimiz olsaydı 'binary' kullanıcaktık.
                                         subset= 'validation')

Found 2275 images belonging to 1 classes.
Found 252 images belonging to 1 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(filters=32,                #Evrişim Ağında kullanılacak filtre sayısı
                 kernel_size=(3,3),         #Filtrenin boyutu
                 padding='same',            #Giriş matriksi ile çıkış matrisi aynı boyutta olmadığından eksik olan pixelleri 0 ile doldurur.Bunun farklı yolları da vardır.
                 input_shape=(224,224,3), #Giriş görüntülerinin boyut bilgisi
                 activation = 'relu'))      #Bu katmandan diğer katmana geçiş için kullanılacak olan aktivasyon fonksiyonu
model.add(MaxPool2D(pool_size=2,strides=(2,2)))
# Bu katmana pooling katmanı denir.Görüntünün özelliklerini kaybetmeden boyutunun küçültülmesine yarar.Maxpooling ise uygulanan filtre içerisindeki e yüksek değerleri alıp sonuç matriksini oluşturmaya yarar.

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation = 'relu'))
model.add(MaxPool2D(pool_size=2,strides=(2,2)))

model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',activation = 'relu'))
model.add(MaxPool2D(pool_size=2,strides=(2,2)))

model.add(Flatten()) # Önceki katmanlardan gelen verileri düzleştirme işlemi yapar. Yani 3D boyutlu veya 2D boyutlu verileri 1D boyutuna indirgeyerek bir sonraki katmanda kullanılmak üzere düzenler.

model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.2))

model.add(Dense(units= 32, activation='relu'))
model.add(Dropout(rate=0.2))

model.add(Dense(units=6, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 28, 28, 32)        0

# Model Optimizasonu

In [ ]:
model.compile(loss='categorical_crossentropy',  #Optimizasyon için kullanılacak olan kayıp fonksiyonu
              optimizer = 'adam',   #Optimize işleminde kullanılacak olan yöntem
              metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), "acc"]) #Başarının nasıl değerlendirileceğinin metrikleri
#Aşırı öğtenmenin karşısına geçme

# Aşırı Öğrenmenin Önüne geçme

In [ ]:
callback = [EarlyStopping(monitor = 'val_loss', #Hangi metrik açısından takip edileceğini belirlenmesi(burada validation_loss kullanıldı)
                          patience = 50, #Kaç adımdan sonra hala ilerleme yoksa durdurma bilgisi girilir
                          verbose = 1, # 0=ekranda hiç bir sonuç verilmez. 1 = Anlık olarak ilerleme barı  2 = Her epochs sonucunda bilgilendirme yapar.
                          mode =  "min"),
            ModelCheckpoint(filepath = 'mymodel.h5', #İlgili dosyanın ne şekilde kaydedileceği belirtilir.
                            monitor = 'val_loss',
                            mode = 'min',
                            save_best_only= True, #Sadece en iyi modeli kaydetme
                            save_weights_only=False)] #Sadece ağırlıkları kaydet

In [ ]:
model.fit_generator(generator = train_generator, #Eğitim veri setini giriyoruz.
                    epochs=100, #Verinin kaç öğrenme sürecinden geçeceğini belirlenir.
                    validation_data = test_generator #Valide edilecek veri setini
                    callbacks = callbacks #Model eğitme sürecinde modelin aşırı öğrenmesinin önüne geçmek amacıyla yapılan bilgilendirme
                    workers=4, #?
                    steps_per_epoch = 2276//32, #?
                    validation_steps = 251//32 #?
                    )

In [ ]:
#------------------
# Accuracy Grafiği
#------------------

plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['acc'], label='Training Accuracy')
plt.plot(history.history['val_acc'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy', fontsize=16)

#--------------
# Loss Grafiği
#--------------

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss', fontsize=16)
plt.show()

In [ ]:
loss, precision, recall, acc = model.evaluate(test_generator, batch_size=32)

In [ ]:
print("\nTest accuracy: %.1f%%" % (100.0 * acc))
print("\nTest loss: %.1f%%" % (100.0 * loss))
print("\nTest precision: %.1f%%" % (100.0 * precision))
print("\nTest recall: %.1f%%" % (100.0 * recall))

In [ ]:
# Clasification Report

x_test, y_test = test.generator.next()

y_pred = model.predict(x_test)

y_pred = np.argmax(y_pred, axis=1)

y_test= np.argmax(y_test, axis=1)

In [ ]:
target_names = list(waste_labels.keys())

In [ ]:
print(classification_report(y_test, y_pred, target_names = target_names))

In [ ]:
# Confusion Matrix

cm = confusion_matrix(y_test, y pred)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
  plt.figure(figsize=(8, 6))
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)
  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
              horizontalalignment="center",
              color="white" if cm[i, j] > thresh else "black")
  plt.tight_layout()
  plt.ylabel('True label', fontweight="bold")
  plt.xlabel('Predicted label', fontweight="bold")

In [ ]:
plot_confusion_matrix(cm, waste_labels.keys(),
                      title = 'Confusion Matrix',
                      cmap = plt.cm.OrRd)